In [4]:
import sys
import os
import numpy as np

# Путь к директории, в которой находится этот файл
# Чтобы можно импортировать папки с кодом из директории выше
# например from censure import Censor
current_directory = os.path.dirname(os.path.abspath(''))
parent_directory = os.path.abspath(os.path.join(current_directory, '..'))
sys.path.append(current_directory)

In [5]:
from scripts.profanity_check import profanity_processing
from scripts.embedding_model import universal_sentence_encoder

2024-09-28 23:17:08.450151: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-28 23:17:08.456501: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-28 23:17:08.524450: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-28 23:17:08.581177: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-28 23:17:08.635882: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been 

In [8]:
responses = [
    "Я люблю свою работу за большую зарплату",
    "Много денег",
    "Хочу много долларов",
    "100 тысяч рублей в секунду",
    "Денежные премии!",
    "Бумажки",
    "Бабосики))",
    "В коллективе царит атмосфера поддержки.",
    "Дружелюбная атмосфера делает работу приятнее.",
    "Мы как друзья, не только коллеги.",
    "Каждый может высказать свое мнение.",
    "Команда — настоящая дружная семья.",
    "Позитивная атмосфера заряжает энергией.",
    "Все готовы поддерживать друг друга.",
    "Мы можем делиться радостями и трудностями.",
    "Вместе справляемся с любыми задачами.",
    "Всегда можно обратиться за помощью.",
    "Еда всегда свежая и аппетитная.",
    "В нашем меню много интересных блюд.",
    "Коллеги любят делиться рецептами.",
    "Проводим обеды с удовольствием и смехом.",
    "Каждую неделю пробуем что-то новое.",
    "Вместе готовим вкусные угощения.",
    "Обсуждаем кулинарные эксперименты после работы.",
    "Мы создаем неповторимые гастрономические пары.",
    "Еда помогает нам лучше узнать друг друга.",
    "Собираемся на праздники за вкусной едой.",
    "Всегда есть возможности для обучения.",
    "Поддерживаем друг друга в развитии.",
    "Обсуждаем новые идеи на митингах.",
    "Курсы и тренинги доступны каждому.",
    "Каждый может предложить свой проект.",
    "Ставим цели и достигаем их вместе.",
    "Обратная связь помогает расти профессионально.",
    "Читаем книги и делимся знаниями.",
    "Нас мотивирует желание стать лучше."
    
]

responses = [response.lower() for response in responses]
responses = np.array(responses)
'; '.join(responses)

'я люблю свою работу за большую зарплату; много денег; хочу много долларов; 100 тысяч рублей в секунду; денежные премии!; бумажки; бабосики)); в коллективе царит атмосфера поддержки.; дружелюбная атмосфера делает работу приятнее.; мы как друзья, не только коллеги.; каждый может высказать свое мнение.; команда — настоящая дружная семья.; позитивная атмосфера заряжает энергией.; все готовы поддерживать друг друга.; мы можем делиться радостями и трудностями.; вместе справляемся с любыми задачами.; всегда можно обратиться за помощью.; еда всегда свежая и аппетитная.; в нашем меню много интересных блюд.; коллеги любят делиться рецептами.; проводим обеды с удовольствием и смехом.; каждую неделю пробуем что-то новое.; вместе готовим вкусные угощения.; обсуждаем кулинарные эксперименты после работы.; мы создаем неповторимые гастрономические пары.; еда помогает нам лучше узнать друг друга.; собираемся на праздники за вкусной едой.; всегда есть возможности для обучения.; поддерживаем друг друга 

In [4]:
model = universal_sentence_encoder()

In [6]:
model.transform(responses)

<tf.Tensor: shape=(36, 512), dtype=float32, numpy=
array([[-0.01534517, -0.04917703,  0.00449831, ..., -0.02355961,
        -0.01564514,  0.0015455 ],
       [ 0.0831308 , -0.00533537, -0.03361337, ...,  0.03192265,
         0.05454447,  0.07531159],
       [ 0.00153435, -0.02513039, -0.03392932, ...,  0.01917938,
         0.06151196,  0.09093355],
       ...,
       [-0.03138888,  0.05251615, -0.03633181, ..., -0.04989275,
         0.00196739,  0.00370043],
       [-0.05370517, -0.01137648,  0.00720926, ..., -0.061986  ,
         0.01351536, -0.03088878],
       [-0.02917297, -0.00114145,  0.00714192, ...,  0.06403864,
         0.01628391,  0.06203831]], dtype=float32)>